__Subject:__ Topics of Tweets of US Congress through the Lens of Campaign Finance

__Date:__ 09/12/2018

__Author:__ Edmund D. Chitwood
***

__Summary:__<br>
<br>The following Notebook
- downloads campaign finance information for each relevant member of Congress from opensecrets.org
***

In [17]:
import pandas as pd
import pickle

In [18]:
congress_metadata = pd.read_pickle('congress_metadata.pkl')

In [20]:
congress_metadata.head()

,last_name,first_name,birthday,gender,type,state,district,senate_class,party,url,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,1952-11-09,M,sen,OH,NaN,1.0,Democrat,https://www.brown.senate.gov,...,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,1958-10-13,F,sen,WA,NaN,1.0,Democrat,https://www.cantwell.senate.gov,...,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,1943-10-05,M,sen,MD,NaN,1.0,Democrat,https://www.cardin.senate.gov,...,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin
3,Carper,Thomas,1947-01-23,M,sen,DE,NaN,1.0,Democrat,https://www.carper.senate.gov/public,...,N00012508,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper
4,Casey,Robert,1960-04-13,M,sen,PA,NaN,1.0,Democrat,https://www.casey.senate.gov,...,N00027503,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.


In [21]:
congress_metadata.columns

Index(['last_name', 'first_name', 'birthday', 'gender', 'type', 'state',
       'district', 'senate_class', 'party', 'url', 'address', 'phone',
       'contact_form', 'rss_url', 'twitter', 'facebook', 'youtube',
       'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'fec_ids', 'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id'],
      dtype='object')

In [18]:
# https://www.opensecrets.org/ is data source for campaign finance data
#!pip install opensecrets-crpapi

In [7]:
from crpapi import CRP
#crp = CRP(apikey)
crp = CRP('21539018e701b26436921b72553a4208')

In [8]:
# Example campaign finance data
contributions = crp.candidates.sector('N00003535')
contributions

[{'@attributes': {'sector_name': 'Agribusiness',
   'sectorid': 'A',
   'indivs': '220786',
   'pacs': '95000',
   'total': '315786'}},
 {'@attributes': {'sector_name': 'Communic/Electronics',
   'sectorid': 'B',
   'indivs': '1336249',
   'pacs': '126000',
   'total': '1462249'}},
 {'@attributes': {'sector_name': 'Construction',
   'sectorid': 'C',
   'indivs': '269334',
   'pacs': '90750',
   'total': '360084'}},
 {'@attributes': {'sector_name': 'Defense',
   'sectorid': 'D',
   'indivs': '11210',
   'pacs': '44000',
   'total': '55210'}},
 {'@attributes': {'sector_name': 'Energy/Nat Resource',
   'sectorid': 'E',
   'indivs': '149317',
   'pacs': '90000',
   'total': '239317'}},
 {'@attributes': {'sector_name': 'Finance/Insur/RealEst',
   'sectorid': 'F',
   'indivs': '2877538',
   'pacs': '968350',
   'total': '3845888'}},
 {'@attributes': {'sector_name': 'Health',
   'sectorid': 'H',
   'indivs': '1521128',
   'pacs': '401200',
   'total': '1922328'}},
 {'@attributes': {'sector_na

In [23]:
# Get Campaign contriubtion information for each member of Congress, broken 
# down by sector and type of donor (i.e. Individual or PAC) 
# Totals are from most recent election cycle

contributions = []

for i in congress_metadata.opensecrets_id:
    try:
        contributions.append(crp.candidates.sector(i))
    except:
        print(i)
        contributions.append('empty')

N00000267


In [28]:
# No data returned for Representative Chris Collins:
# https://www.opensecrets.org/members-of-congress/summary?cid=N00001285&cycle=2018
# Collins suspended his campaign on August 11, 2018 after he was indicted for
# insider trading 
# Drop him from data

contributions.remove('empty')
congress_metadata.drop(congress_metadata[congress_metadata.opensecrets_id\
                       =='N00000267'].index, inplace=True)

In [35]:
# JSON objects for each member of Congress vary
# For most Congress members, JSON object contain info for 13 sectors
# That is not the case for the following
for v in contributions:
    if len(v) != 13:
        print(len(v))

12
12
12
12
12
12
9
12
12
12
11
12
12
12
12
3
12
1
12
10
11
12
12
12
12
12
12
9
12
12
10
10
12
11
12
11
12
12
12
10
11
11
12
12
12
12
12
11
6
12
12
12
12
12
12
12
12
12
11
12
12
11
8
12
12
11
12
12
12
12
10


In [42]:
# Flatten list of dicts of dicts
flat_contributions = []

for i,v in enumerate(contributions):
    if type(v) == list:
        for j in range(len(v)):
            contributions[i][j]['@attributes']['os_id'] = ids[i]
            flat_contributions.append(contributions[i][j]['@attributes'])
    else:
        v['@attributes']['os_id'] = ids[i]
        flat_contributions.append(v['@attributes'])

In [43]:
flat_contributions[0]

{'sector_name': 'Agribusiness',
 'sectorid': 'A',
 'indivs': '220786',
 'pacs': '95000',
 'total': '315786',
 'os_id': 'N00003535'}

In [44]:
contributions_congress = pd.DataFrame(flat_contributions)

In [45]:
contributions_congress.shape

(6477, 6)

In [46]:
contributions_congress.describe()

,indivs,os_id,pacs,sector_name,sectorid,total
count,6477,6477,6477,6477,6477,6477
unique,4372,508,2366,14,14,5274
top,0,N00000575,0,Finance/Insur/RealEst,F,1000
freq,689,13,380,507,507,44


In [47]:
contributions_congress.to_pickle('contributions_congress.pkl')